In [ ]:
# Extract keyords and important topics
# Wordcloud to show the ones that occur the most often
# Visualizations between employee title, tenure, department and the rating they gave

In [61]:
# Heatmap on a map of the world for grades by location

In [62]:
# Extract the top N positives and negative points that come up in reviews

In [ ]:
# General sentiment about the company - different segments of the employee population.

# > What are the segments of people we are interested in? Everyone? Mangers and up?
# > What do we want to know about them? How they feel about the company
# > Why do they feel that way? - Topic Extraction and Clustering
# > How has this changed through time? Were people giving better reviews in 2018?

In [48]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [49]:
data = pd.read_csv("Data.csv")
data.head()

,Unnamed: 0,Title,Rating,Designation,Pros,Cons,Author Info
0,0,Great Culture and Company That Cares,4,Current Employee,Not much politics and very few ego's. Everyone...,Need to focus on career growth and promoting w...,13 Jan 2021 - Senior Corporate Recruiter in Tr...
1,1,Shaping the Future of Mobility!,5,"Current Employee, more than 1 year","Global Work Environment, Cutting Edge Technolo...",None to mention at this time.,"14 Apr 2021 - Program Manager in Dublin, Dublin"
2,2,Lots of potential- getting there,4,"Current Employee, more than 1 year",A very fast paced environment - you definitely...,Training and coaching could be better- high ex...,"22 May 2021 - SCM in Dublin, Dublin"
3,3,Alright but not the best,3,"Current Employee, more than 1 year","Excellent office, good staff, good pay, good b...","Dis organised, office politics at mid/top leve...","8 Mar 2021 - Business Analyst in Dublin, Dublin"
4,4,Aptiv,3,Current Employee,It is one of the leaders in the automotive ind...,"High pressure for hitting numbers, it is more ...","7 Nov 2020 - Supply Chain in Dublin, Dublin"


In [50]:
data.nunique()

Unnamed: 0     579
Title          503
Rating           5
Designation     14
Pros           544
Cons           552
Author Info    562
dtype: int64

In [51]:
### Cleaning the data

In [52]:
data.isnull().sum()

Unnamed: 0     0
Title          0
Rating         0
Designation    0
Pros           0
Cons           0
Author Info    0
dtype: int64

In [53]:
# drop redundant data
data_cleaned = data.drop("Unnamed: 0", 1, inplace=True)
print(data_cleaned)

None


In [54]:
# checking for outliers - data point that differ significantly


In [60]:
pd['Rating']=pd['Rating'].astype(int)

TypeError: 'module' object is not subscriptable